In [1]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("Instagram_Analytics.csv")
print(df.shape)  

(29999, 15)


Additionally adding two columns- Account id, followers_count(before post) for an effective analysis

Decide number of creators & posts distribution

In [3]:
NUM_CREATORS = 300
TOTAL_POSTS = len(df)

# Generate creator IDs
creator_ids = [f"creator_{i:03d}" for i in range(1, NUM_CREATORS + 1)]


Create random post counts per creator (80–120):

In [12]:
NUM_CREATORS = 300
TOTAL_POSTS = len(df)

np.random.seed(42)

# Step 1: initial realistic range
posts_per_creator = np.random.randint(80, 121, size=NUM_CREATORS)

# Step 2: scale proportionally
scaled = posts_per_creator / posts_per_creator.sum() * TOTAL_POSTS
posts_per_creator = np.floor(scaled).astype(int)

# Step 3: distribute remainder safely
remainder = TOTAL_POSTS - posts_per_creator.sum()

indices = np.argsort(posts_per_creator)

for i in range(remainder):
    posts_per_creator[indices[i % NUM_CREATORS]] += 1

# Final sanity checks
assert posts_per_creator.sum() == TOTAL_POSTS
assert posts_per_creator.max() <= 125  # safe upper bound


In [6]:
print(sum(posts_per_creator), len(df))


29999 29999


Assign creators to rows (row assignment)

In [13]:
account_id_column = []

for creator, count in zip(creator_ids, posts_per_creator):
    account_id_column.extend([creator] * count)

# Final safety check
assert len(account_id_column) == TOTAL_POSTS

df["account_id"] = account_id_column



In [14]:
df["account_id"].value_counts().describe()


count    300.000000
mean      99.996667
std       11.719834
min       80.000000
25%       90.000000
50%      102.000000
75%      110.250000
max      119.000000
Name: count, dtype: float64

Sort posts per creator by date

In [15]:
df["upload_date"] = pd.to_datetime(df["upload_date"])

df = df.sort_values(by=["account_id", "upload_date"]).reset_index(drop=True)


Assign initial followers per creator

In [16]:
initial_followers_map = {}

for creator in creator_ids:
    initial_followers_map[creator] = np.random.choice(
        [5000, 15000, 30000, 60000, 100000],
        p=[0.30, 0.25, 0.20, 0.15, 0.10]
    )


Generate followers BEFORE each post

In [17]:
df["followers_count_before_post"] = 0

for creator in creator_ids:
    mask = df["account_id"] == creator
    creator_df = df.loc[mask].copy()

    followers_before = []
    current_followers = initial_followers_map[creator]

    for gained in creator_df["followers_gained"]:
        followers_before.append(current_followers)
        current_followers += gained

    df.loc[mask, "followers_count_before_post"] = followers_before


In [19]:
df[["account_id", "followers_gained", "followers_count_before_post"]].head(10)
df.head(8)

,post_id,upload_date,media_type,likes,comments,shares,saves,reach,impressions,caption_length,hashtags_count,followers_gained,traffic_source,engagement_rate,content_category,account_id,followers_count_before_post
0,IG0000097,2024-11-21 09:25:22.954916,Photo,161614,6644,2604,14819,30428,411145,1207,14,534,External,45.16,Comedy,creator_001,15000
1,IG0000100,2024-11-21 09:25:22.954916,Video,192838,9209,3916,4460,891372,1179545,1233,12,335,External,17.84,Beauty,creator_001,15534
2,IG0000041,2024-11-25 09:25:22.954916,Video,78059,5416,419,8377,1316621,1752540,272,3,359,External,5.26,Comedy,creator_001,15869
3,IG0000027,2024-11-29 09:25:22.954916,Carousel,198903,709,2532,7163,1610322,1655482,155,9,925,Reels Feed,12.64,Photography,creator_001,16228
4,IG0000001,2024-11-30 09:25:22.954916,Reel,31627,7559,4530,6393,615036,1007750,1340,3,899,Home Feed,4.97,Technology,creator_001,17153
5,IG0000037,2024-12-01 09:25:22.954916,Reel,51285,7705,3797,2518,1209734,1563306,700,13,336,Reels Feed,4.18,Comedy,creator_001,18052
6,IG0000016,2024-12-06 09:25:22.954916,Reel,180946,4638,3852,2095,1713467,2211458,739,12,752,Hashtags,8.66,Music,creator_001,18388
7,IG0000048,2024-12-08 09:25:22.954916,Carousel,92643,1642,160,12118,312299,510800,844,7,335,Home Feed,20.86,Fitness,creator_001,19140


In [25]:
df.head(10)

,post_id,upload_date,media_type,likes,comments,shares,saves,reach,impressions,caption_length,hashtags_count,followers_gained,traffic_source,engagement_rate,content_category,account_id,followers_count_before_post
0,IG0000854,2025-10-31 09:25:22.954916,Video,84641,8862,2388,6386,17372,27727,1985,6,77,External,368.87,Lifestyle,creator_009,142041
1,IG0013402,2025-04-04 09:25:22.954916,Reel,132420,5003,3667,5601,1546640,2016802,1750,6,171,Reels Feed,7.27,Technology,creator_135,54355
2,IG0010751,2025-04-19 09:25:22.954916,Video,31476,6754,4382,12891,1916933,2079234,1502,26,892,Hashtags,2.67,Photography,creator_109,26763
3,IG0013361,2025-08-15 09:25:22.954916,Video,94556,9750,1667,10965,1451049,1816268,626,27,530,Hashtags,6.44,Fashion,creator_135,71457
4,IG0027034,2025-02-27 09:25:22.954916,Video,38105,1743,4306,2341,1240598,1707533,1131,16,289,Explore,2.72,Technology,creator_271,119968
5,IG0022212,2025-09-05 09:25:22.954916,Carousel,113141,9477,3079,310,496673,690055,26,4,604,Profile,18.26,Photography,creator_223,76806
6,IG0027062,2024-11-25 09:25:22.954916,Video,39301,9597,763,369,222449,692071,447,25,295,Reels Feed,7.23,Photography,creator_271,102513
7,IG0013721,2025-06-20 09:25:22.954916,Video,72287,1077,4039,3288,61295,421707,635,15,513,Profile,19.13,Photography,creator_138,39573
8,IG0021589,2025-09-03 09:25:22.954916,Photo,60627,2747,3371,4244,841674,860179,1397,12,550,External,8.25,Technology,creator_218,78277
9,IG0012768,2025-03-30 09:25:22.954916,Video,171776,8005,3337,837,1637998,2055998,837,12,16,Home Feed,8.95,Fitness,creator_129,27157


In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df["account_id"].head(10)


0    creator_009
1    creator_135
2    creator_109
3    creator_135
4    creator_271
5    creator_223
6    creator_271
7    creator_138
8    creator_218
9    creator_129
Name: account_id, dtype: object

In [28]:
df.dtypes


post_id                                object
upload_date                    datetime64[ns]
media_type                             object
likes                                   int64
comments                                int64
shares                                  int64
saves                                   int64
reach                                   int64
impressions                             int64
caption_length                          int64
hashtags_count                          int64
followers_gained                        int64
traffic_source                         object
engagement_rate                       float64
content_category                       object
account_id                             object
followers_count_before_post             int64
dtype: object

In [ ]:
df["post_id"].duplicated().sum()
df.isna().sum().sort_values(ascending=False)

post_id                        0
caption_length                 0
account_id                     0
content_category               0
engagement_rate                0
traffic_source                 0
followers_gained               0
hashtags_count                 0
impressions                    0
upload_date                    0
reach                          0
saves                          0
shares                         0
comments                       0
likes                          0
media_type                     0
followers_count_before_post    0
dtype: int64

In [35]:
df = df[df["impressions"] >= df["reach"]]
df = df[df["followers_gained"] <= df["reach"]]


In [34]:
total_engagement = df["likes"] + df["comments"] + df["shares"] + df["saves"]

df = df[total_engagement <= df["reach"]]


In [37]:
df["engagement_rate_clean"] = (
    (df["likes"] + df["comments"] + df["shares"] + df["saves"]) /
    df["followers_count_before_post"]
)
df.drop(columns=["engagement_rate"], inplace=True)
df.rename(columns={"engagement_rate_clean": "engagement_rate"}, inplace=True)


In [38]:
upper_cap = df["engagement_rate"].quantile(0.99)
df["engagement_rate"] = df["engagement_rate"].clip(upper=upper_cap)


In [39]:
df.describe(include="all")


,post_id,upload_date,media_type,likes,comments,shares,saves,reach,impressions,caption_length,hashtags_count,followers_gained,traffic_source,content_category,account_id,followers_count_before_post,engagement_rate
count,28236,28236,28236,28236.000000,28236.000000,28236.000000,28236.000000,2.823600e+04,2.823600e+04,28236.000000,28236.000000,28236.000000,28236,28236,28236,28236.000000,28236.000000
unique,28236,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,10,300,NaN,NaN
top,IG0013402,NaN,Reel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home Feed,Fashion,creator_152,NaN,NaN
freq,1,NaN,7085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4773,2871,114,NaN,NaN
mean,NaN,2025-05-19 18:22:08.649773824,NaN,98181.831138,5014.319875,2499.010058,7478.154165,1.054345e+06,1.304715e+06,1102.458457,14.890849,501.661886,NaN,NaN,NaN,54126.576144,3.279377
min,NaN,2024-11-19 09:25:22.954916,NaN,7.000000,0.000000,0.000000,0.000000,2.091300e+04,3.118100e+04,0.000000,0.000000,0.000000,NaN,NaN,NaN,5000.000000,0.055499
25%,NaN,2025-02-18 09:25:22.954916096,NaN,48112.250000,2522.750000,1242.000000,3671.750000,5.806265e+05,8.324002e+05,555.000000,7.000000,250.000000,NaN,NaN,NaN,30672.750000,1.162923
50%,NaN,2025-05-18 09:25:22.954916096,NaN,96940.500000,5035.000000,2492.000000,7467.000000,1.051291e+06,1.299030e+06,1099.500000,15.000000,500.000000,NaN,NaN,NaN,46936.500000,2.216262
75%,NaN,2025-08-18 09:25:22.954916096,NaN,147928.750000,7520.000000,3765.250000,11282.250000,1.525056e+06,1.776869e+06,1655.000000,23.000000,755.000000,NaN,NaN,NaN,68475.250000,3.969076
max,NaN,2025-11-19 09:25:22.954916,NaN,200000.000000,10000.000000,5000.000000,15000.000000,1.999865e+06,2.497940e+06,2200.000000,30.000000,1000.000000,NaN,NaN,NaN,165591.000000,21.070199


In [41]:
df["post_hour"] = df["upload_date"].dt.hour
df["post_day"] = df["upload_date"].dt.day_name()
df["is_weekend"] = df["upload_date"].dt.weekday >= 5


In [42]:
df["has_high_hashtags"] = df["hashtags_count"] >= 10
df["long_caption"] = df["caption_length"] >= 150


In [43]:
df.head()

,post_id,upload_date,media_type,likes,comments,shares,saves,reach,impressions,caption_length,...,traffic_source,content_category,account_id,followers_count_before_post,engagement_rate,post_hour,post_day,is_weekend,has_high_hashtags,long_caption
1,IG0013402,2025-04-04 09:25:22.954916,Reel,132420,5003,3667,5601,1546640,2016802,1750,...,Reels Feed,Technology,creator_135,54355,2.698758,9,Friday,False,False,True
2,IG0010751,2025-04-19 09:25:22.954916,Video,31476,6754,4382,12891,1916933,2079234,1502,...,Hashtags,Photography,creator_109,26763,2.073871,9,Saturday,True,True,True
3,IG0013361,2025-08-15 09:25:22.954916,Video,94556,9750,1667,10965,1451049,1816268,626,...,Hashtags,Fashion,creator_135,71457,1.636481,9,Friday,False,True,True
4,IG0027034,2025-02-27 09:25:22.954916,Video,38105,1743,4306,2341,1240598,1707533,1131,...,Explore,Technology,creator_271,119968,0.387562,9,Thursday,False,True,True
5,IG0022212,2025-09-05 09:25:22.954916,Carousel,113141,9477,3079,310,496673,690055,26,...,Profile,Photography,creator_223,76806,1.640588,9,Friday,False,False,False


EDA

volume by media type

In [44]:
df["media_type"].value_counts(normalize=True) * 100


media_type
Reel        25.092081
Carousel    25.081456
Video       25.010625
Photo       24.815838
Name: proportion, dtype: float64

Engagement rate distribution

In [45]:
df["engagement_rate"].describe()


count    28236.000000
mean         3.279377
std          3.517044
min          0.055499
25%          1.162923
50%          2.216262
75%          3.969076
max         21.070199
Name: engagement_rate, dtype: float64

Engagement by media type

In [46]:
df.groupby("media_type")["engagement_rate"].mean().sort_values(ascending=False)


media_type
Reel        3.312636
Video       3.293429
Photo       3.257083
Carousel    3.254150
Name: engagement_rate, dtype: float64

Saves & shares vs likes

In [47]:
df["high_intent_engagement"] = df["saves"] + df["shares"]

df.groupby("media_type")["high_intent_engagement"].mean()


media_type
Carousel     9985.571731
Photo        9959.551877
Reel         9949.033451
Video       10014.430473
Name: high_intent_engagement, dtype: float64

Best posting hour

In [48]:
df.groupby("post_hour")["engagement_rate"].mean()


post_hour
9    3.279377
Name: engagement_rate, dtype: float64

In [49]:
df.groupby("is_weekend")["engagement_rate"].mean()


is_weekend
False    3.260059
True     3.328569
Name: engagement_rate, dtype: float64

Reach vs engagement

In [50]:
df[["reach", "engagement_rate"]].corr()


,reach,engagement_rate
reach,1.000000,0.023645
engagement_rate,0.023645,1.000000


Followers gained vs traffic source

In [51]:
df.groupby("traffic_source")["followers_gained"].mean().sort_values(ascending=False)


traffic_source
Profile       510.231097
Explore       504.075443
Home Feed     502.635240
External      501.593041
Reels Feed    499.495772
Hashtags      492.119529
Name: followers_gained, dtype: float64

Engagement rate vs followers (scale effect)

In [52]:
df["follower_bucket"] = pd.qcut(
    df["followers_count_before_post"],
    q=4,
    labels=["Small", "Medium", "Large", "Very Large"]
)

df.groupby("follower_bucket")["engagement_rate"].mean()


C:\Users\1464y\AppData\Local\Temp\ipykernel_9296\1906423377.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("follower_bucket")["engagement_rate"].mean()


follower_bucket
Small         6.957431
Medium        2.962009
Large         2.021369
Very Large    1.176700
Name: engagement_rate, dtype: float64

In [53]:
df.head()

,post_id,upload_date,media_type,likes,comments,shares,saves,reach,impressions,caption_length,...,account_id,followers_count_before_post,engagement_rate,post_hour,post_day,is_weekend,has_high_hashtags,long_caption,high_intent_engagement,follower_bucket
1,IG0013402,2025-04-04 09:25:22.954916,Reel,132420,5003,3667,5601,1546640,2016802,1750,...,creator_135,54355,2.698758,9,Friday,False,False,True,9268,Large
2,IG0010751,2025-04-19 09:25:22.954916,Video,31476,6754,4382,12891,1916933,2079234,1502,...,creator_109,26763,2.073871,9,Saturday,True,True,True,17273,Small
3,IG0013361,2025-08-15 09:25:22.954916,Video,94556,9750,1667,10965,1451049,1816268,626,...,creator_135,71457,1.636481,9,Friday,False,True,True,12632,Very Large
4,IG0027034,2025-02-27 09:25:22.954916,Video,38105,1743,4306,2341,1240598,1707533,1131,...,creator_271,119968,0.387562,9,Thursday,False,True,True,6647,Very Large
5,IG0022212,2025-09-05 09:25:22.954916,Carousel,113141,9477,3079,310,496673,690055,26,...,creator_223,76806,1.640588,9,Friday,False,False,False,3389,Very Large


In [54]:
df.groupby("has_high_hashtags")["engagement_rate"].mean()


has_high_hashtags
False    3.294906
True     3.271727
Name: engagement_rate, dtype: float64

YData Profiling(an automated data audit that checks distributions, correlations, missing values, and anomalies.)

In [55]:
from ydata_profiling import ProfileReport
profile = ProfileReport(
    df,
    title="Instagram Analytics Dataset - Data Profiling Report",
    explorative=True,
    minimal=False
)
profile.to_file("instagram_ydata_profiling.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\1464y\AppData\Local\Programs\Python\Python312\Lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 4 4 4]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
df.to_csv("instagram_analytics_cleaned.csv", index=False)
